In [1]:
pip install geopandas osmnx folium shapely


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 57.7 MB/s eta 0:00:00
  Attempting uninstall: geopandas
    Found existing installation: geopandas 1.0.1
    Uninstalling geopandas-1.0.1:
      Successfully uninstalled geopandas-1.0.1


In [2]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point, Polygon
import requests


In [3]:
# Sample POI Data
data = {
    'name': ['Restaurant A', 'Coffee Shop B', 'Museum C'],
    'type': ['Restaurant', 'Cafe', 'Museum'],
    'latitude': [12.9716, 12.9352, 12.9756],
    'longitude': [77.5946, 77.6124, 77.6092]
}

# Creating a DataFrame
poi_df = pd.DataFrame(data)

# Convert DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(poi_df['longitude'], poi_df['latitude'])]
gdf = gpd.GeoDataFrame(poi_df, geometry=geometry, crs='EPSG:4326') # WGS 84 coordinate system

# Display the first few rows
print(gdf.head())


            name        type  latitude  longitude                   geometry
0   Restaurant A  Restaurant   12.9716    77.5946  POINT (77.59460 12.97160)
1  Coffee Shop B        Cafe   12.9352    77.6124  POINT (77.61240 12.93520)
2       Museum C      Museum   12.9756    77.6092  POINT (77.60920 12.97560)


In [4]:
# Initialize a Folium map centered around the coordinates
map_center = [12.9716, 77.5946] # Centering around a known location
poi_map = folium.Map(location=map_center, zoom_start=12)

# Add POIs to the map
for _, row in gdf.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']} - {row['type']}",
        icon=folium.Icon(color='blue')
    ).add_to(poi_map)

# Display the map
poi_map


In [5]:
def fetch_osm_data(latitude, longitude, radius=1000):
    """
    Fetches POI data from OpenStreetMap using Overpass API.
    """
    overpass_url = "http://overpass-api.de/api/interpreter"
    overpass_query = f"""
    [out:json];
    (
      node(around:{radius},{latitude},{longitude})["amenity"];
    );
    out body;
    """
    response = requests.get(overpass_url, params={'data': overpass_query})
    data = response.json()
    return data

# Example: Fetch POIs around the central coordinates
osm_data = fetch_osm_data(12.9716, 77.5946)
print(len(osm_data['elements']), "POIs fetched from OSM.")


498 POIs fetched from OSM.


In [6]:
# Extract relevant details from the fetched OSM data
osm_poi_list = []
for element in osm_data['elements']:
    if 'tags' in element:
        name = element['tags'].get('name', 'Unnamed')
        poi_type = element['tags'].get('amenity', 'Unknown')
        lat = element['lat']
        lon = element['lon']
        osm_poi_list.append([name, poi_type, lat, lon])

# Create DataFrame from the extracted list
osm_poi_df = pd.DataFrame(osm_poi_list, columns=['name', 'type', 'latitude', 'longitude'])

# Convert to GeoDataFrame
osm_geometry = [Point(xy) for xy in zip(osm_poi_df['longitude'], osm_poi_df['latitude'])]
osm_gdf = gpd.GeoDataFrame(osm_poi_df, geometry=osm_geometry, crs='EPSG:4326')

# Display the first few rows
print(osm_gdf.head())


                         name        type   latitude  longitude  \
0                     Koshy's  restaurant  12.975632  77.601476   
1                      Konārk  restaurant  12.968810  77.602143   
2   Old Madras Baking Company        cafe  12.969738  77.600377   
3  Cubbon Park Police Station      police  12.976240  77.598346   
4              Hard Rock Cafe  restaurant  12.976226  77.601481   

                    geometry  
0  POINT (77.60148 12.97563)  
1  POINT (77.60214 12.96881)  
2  POINT (77.60038 12.96974)  
3  POINT (77.59835 12.97624)  
4  POINT (77.60148 12.97623)  


In [7]:
# Calculate distances between each original POI and the closest OSM POI
def calculate_nearest_distance(row, osm_gdf):
    """
    Calculate the distance to the nearest OSM POI.
    """
    distances = osm_gdf.geometry.distance(row['geometry'])
    return distances.min()

# Apply the distance calculation
gdf['nearest_osm_distance'] = gdf.apply(calculate_nearest_distance, osm_gdf=osm_gdf, axis=1)

# Display the DataFrame with the calculated distances
print(gdf[['name', 'type', 'nearest_osm_distance']])


            name        type  nearest_osm_distance
0   Restaurant A  Restaurant              0.000256
1  Coffee Shop B        Cafe              0.031964
2       Museum C      Museum              0.006157


<ipython-input-7-f7f715dbd724>:6: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = osm_gdf.geometry.distance(row['geometry'])


In [8]:
# Visualize discrepancies on the map
for _, row in gdf.iterrows():
    color = 'green' if row['nearest_osm_distance'] < 100 else 'red'  # Red for larger discrepancies
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['name']} - {row['type']}, Distance: {row['nearest_osm_distance']:.2f} meters",
        icon=folium.Icon(color=color)
    ).add_to(poi_map)

# Display the updated map
poi_map
